# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os


# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
citiesDF = pd.read_csv("output_data/cities.csv")
del citiesDF["Unnamed: 0"]
citiesDF.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


In [ ]:
# Store latitude and longitude in locations & Humidity   -> for Weight
locations = citiesDF[["Latitude","Longitude"]]
humidList = citiesDF["Humidity"]

In [ ]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# CREATE heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidList, dissipating = False, max_intensity =1000, point_radius =5)

#Add layer
fig.add_layer(heat_layer)

#display
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
# Temperature between 26° and 32° Celcius
# Wind speed less than 8 mph.
# Zero cloudiness.

vacTemp = citiesDF.loc[(citiesDF["Temperature"]>26) & (citiesDF["Temperature"]<32)&(citiesDF["Cloudiness"]==0)&(citiesDF["Wind Speed"]<8)]
#Drop na values
vacTemp.dropna()
vacTemp.head()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = vacTemp[["City ID Number","City","Latitude","Longitude","Temperature","Humidity","Cloudiness","Wind Speed"]]
hotel_df.insert(8,"Hotel Name","")
hotel_df.insert(9,"Country","")
hotel_df

In [ ]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels near city
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    latC = row["Latitude"]
    lngC = row["Longitude"]

    # change city each iteration while leaving original params in place
    params["location"] = f"{latC},{lngC}"

    # Use the keyword, search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    responseInfo = requests.get(base_url, params=params)
    responseInfo = responseInfo.json()

    #Get the first hotel found
    counter = 0
    for place in responseInfo["results"]:
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
        try:
            hotel_df.loc[index, "Hotel Name"] = responseInfo["results"][0]["name"]
            hotel_df.loc[index, "Country"] = responseInfo["results"][0]["place_id"]
            counter += 1
            if counter == 1:
                break
        except (KeyError, IndexError):
            print("Hotel information not found. Skipping...")
        
hotel_df

# This section not worked, it was intended to find Country Name
DOCUMENTATION SHOWED THE URL USED IN THIS SECTION AS THE ONE TO GET THE DETAILS OF THE ADDRESS, 
BUT THE JSON RESPONSE DID NOT SHOW THAT INFO
SOURCE: https://developers.google.com/places/web-service/details#PlaceDetailsResponses

--> params dictionary to update each iteration, trying to find COUNTRY
params = {"key": g_key}

--> Loop through the hotel_df and run PLACEID search for each Country
for index, row in hotel_df.iterrows():
    placeId = row["Country"]
    params['place_id'] = f"{placeId}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"
    responseCountry = requests.get(base_url, params=params)
    responseCountry = responseCountry.json()
    #print(json.dumps(responseInfo, indent=4, sort_keys=True))    
   
    hotel_df.loc[index, "Country"] = hotel_df["result"][0][6]["long_name"]
    hotel_df.head()

In [ ]:
locations2 = hotel_df[["Latitude", "Longitude"]]
humid2 = hotel_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations2, weights=humid2, dissipating = False, max_intensity =1000, point_radius =10)
fig.add_layer(heat_layer)
fig

In [ ]:
narrowed_city_df = hotel_df[["City","Hotel Name","Country"]]
# Visualize
print(len(narrowed_city_df)) 
narrowed_city_df.head()

In [ ]:
cityR = row["City"]
hotelR = row["Hotel Name"]
countryR = row["Country"]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=(hotel_info))
fig.add_layer(markers)
fig

# Display Map